![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Cleaning numerical data

For this lab, we will be using the dataset in the Customer Analysis Business Case. This dataset can be found in `files_for_lab` folder.

## Context

An auto insurance company has collected some data about its customers including their _demographics_, _education_, _employment_, _policy details_, _vehicle information_ on which insurance policy is, and _claim amounts_. You will help the senior management with some business questions that will help them to better understand their customers, improve their services, and improve profitability.

**Some business Objectives**:

- Retain customers,
- Analyze relevant customer data,
- Develop focused customer retention programs.

Based on the analysis, take targeted actions to increase profitable customer response, retention, and growth.

## Instructions

### 1. Import the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from functions import lowercase_cols

### 2. Load the we_fn_use_c_marketing_customer_value_analysis.csv into the variable customer_df

In [ ]:
customer_df = pd.read_csv("files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")

### 3. First look at its main features (head, shape, info).

In [ ]:
customer_df.head()

In [ ]:
customer_df.shape

In [ ]:
customer_df.info()

### 4. Rename the columns so they follow the PE8 (snake case) (lower_case_with_underscores)

In [ ]:
customer_df = lowercase_cols(customer_df)
customer_df.columns

In [ ]:
customer_df.rename(columns = {"employmentstatus":"employment_status"}, inplace = True)
customer_df.columns

### 5. Change effective to date column to datetime format.

In [ ]:
customer_df["effective_to_date"] = pd.to_datetime(customer_df["effective_to_date"])
customer_df["effective_to_date"].info()

### 6. Define a function that differentiates between continuous and discrete variables.

Hint: The number of unique values might be useful. Store continuous data into a continuous_df variable and do the same for discrete_df

In [ ]:
# Let's check how many unique values each numeric column has

for col in customer_df.select_dtypes(np.number):
    print(col, "-", customer_df[col].nunique())

Looks like there are 3 pretty clear continuous data columns with more than 5000 different values.

We can use this threshold in our function below, but we could set a lower threshold just in case.

Or just define an inputable threshold in our function, for example:

In [ ]:
len(customer_df)*0.1

In [ ]:
def continuous_vs_discrete(df, threshold = 0.1):
    
    df2 = df.select_dtypes(np.number)
    df3 = df.select_dtypes(np.number)
    
    df2 = df2.loc[:,df2.nunique()>=len(df)*threshold]
    df3 = df3.loc[:,df3.nunique()<len(df)*threshold]
        
    return df2, df3

customer_continuous, customer_discrete = continuous_vs_discrete(customer_df)              

In [ ]:
customer_continuous

In [ ]:
customer_discrete

### 7. Plot a correlation matrix, comment what you see.

In [ ]:
sns.heatmap(customer_df.corr(), annot = True)

Overall there is no correlation between columns, only destacable one is a 0.63 correlation betweel columns `monthly_premium_auto` and `total_claim_amount`.

There are also 3 more correlation higher than the average, which are:
- `customer_lifetime_value` - `monthly_premium auto`
- `customer_lifetime_value` - `total_claim_amount`
- `income` - `total_claim_amount`

### 8.1 Create a function to plot every discrete variable.

In [ ]:
def hist_maker(df):
    '''Returns histplots in a single fig for each column of a given DataFrame
    
    Input: DataFrame
    Output: Histplots of all the columns'''
    
    cols = list(df.columns)
    x = len(cols)
    fig, ax = plt.subplots(1,x, figsize=(20,10))

    for col in cols:
        y = cols.index(col)
        sns.histplot(data=df, x=col, ax = ax[y])
        ax[y].set_title(col)
        
    return

In [ ]:
hist_maker(customer_discrete)

### 8.2 Do the same with the continuous variables

(Be Careful, you may need to change the plot type to one better suited for continuous data!)

In [ ]:
hist_maker(customer_continuous)

In [ ]:
customer_continuous

In [ ]:
def kdeplot_maker(df):
    '''Returns KDEplots in a single fig for each column of a given DataFrame
    
    Input: DataFrame
    Output: KDEplots of all the columns'''
	
    cols = list(df.columns)
    x = len(cols)
    fig, ax = plt.subplots(1,x, figsize=(20,10))
    
    for col in cols:
        y = cols.index(col)
        sns.kdeplot(data=df, x=col, ax = ax[y])
        ax[y].set_title(col)
        
    return

In [ ]:
kdeplot_maker(customer_continuous)

### 9. Comment what you can see in the plots.

Data appears to be right skewed on both discrete and continuous columns.

No outliers in the `income` column.

Balanced data in the `months_since_policy_inception`.

### 10. Look for outliers in the continuous variables.

(HINT: There’s a good plot to do that!)

In [ ]:
def box_maker(df):
    '''Returns boxplots in a single fig for each column of a given DataFrame
    
    Input: DataFrame
    Output: Boxplot of all the columns'''
	
    cols = list(df.columns)
    x = len(cols)
    fig, ax = plt.subplots(1,x, figsize=(20,10))
    
    for col in cols:
        y = cols.index(col)
        sns.boxplot(data=df, y=col, ax = ax[y])
        ax[y].set_title(col)
        
    return

box_maker(customer_continuous)

### 11. Did you find outliers?  Comment what you will do with them.

There are outliers in both the `customer_lifetime_value` and `total_claim¨_amount`

We can drop them using the iqr.

In [ ]:
iqr_clv = np.nanpercentile(customer_continuous["customer_lifetime_value"], 75) - np.nanpercentile(customer_continuous["customer_lifetime_value"], 25)
upper_limit_clv = np.nanpercentile(customer_continuous["customer_lifetime_value"], 75) + (1.5 * iqr_clv)
print(upper_limit_clv)

len(customer_continuous[customer_continuous["customer_lifetime_value"] > upper_limit_clv])

In [ ]:
iqr_tca = np.nanpercentile(customer_continuous["total_claim_amount"], 75) - np.nanpercentile(customer_continuous["total_claim_amount"], 25)
upper_limit_tca = np.nanpercentile(customer_continuous["total_claim_amount"], 75) + (1.5 * iqr_tca)
print(upper_limit_tca)

len(customer_continuous[customer_continuous["total_claim_amount"] > upper_limit_tca])

In [ ]:
customer_df = customer_df[customer_df["customer_lifetime_value"] < upper_limit_clv]
customer_df = customer_df[customer_df["total_claim_amount"] < upper_limit_tca]

customer_df

### 12. Check all columns for NaN values.

Decide what (if anything) you will need to do with them.

In [ ]:
customer_df.isna().sum()

There are no NAs.

In [ ]:
customer_df.to_csv("customer.csv")